In [41]:
import math

class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0
        self.label = label
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

In [47]:
import random

class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

class Neuron(Module):

    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin = nonlin

    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP(Module):

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

In [48]:
import pandas as pd

road_accident_dataset = pd.read_csv("dataset/road_accident_data.csv")

In [49]:
def build_training_dataset():
    training_data = []
    labelled_data = []

    for road_accident_data in road_accident_dataset.itertuples():
        age = Value(float(road_accident_data.age)/100.0)
        gender = Value(float(road_accident_data.gender))
        speed_of_impact = Value(float(road_accident_data.speed_of_impact)/100.0)
        helmet_used = Value(float(road_accident_data.helmet_used))
        seatbelt_used = Value(float(road_accident_data.seatbelt_used))

        training_data.append([age, gender, speed_of_impact, helmet_used, seatbelt_used])
        # print(f"{type(age.data)}, {type(gender.data)},{type(speed_of_impact.data)}, {type(helmet_used.data)}, {type(seatbelt_used.data)}")
        survived = Value(float(-1.0 if road_accident_data.survived == 0 else 1.0))
        # print(age.grad)
        labelled_data.append(survived)
    
    return training_data, labelled_data

def build_raw_training_dataset():
    training_data = []
    labelled_data = []

    for road_accident_data in road_accident_dataset.itertuples():
        age = float(road_accident_data.age)/100.0
        gender = float(road_accident_data.gender)
        speed_of_impact = float(road_accident_data.speed_of_impact)/100.0
        helmet_used = float(road_accident_data.helmet_used)
        seatbelt_used = float(road_accident_data.seatbelt_used)

        training_data.append([age, gender, speed_of_impact, helmet_used, seatbelt_used])
        # print(f"{type(age.data)}, {type(gender.data)},{type(speed_of_impact.data)}, {type(helmet_used.data)}, {type(seatbelt_used.data)}")
        survived = float(-1.0 if road_accident_data.survived == 0 else 1.0)
        # print(age.grad)
        labelled_data.append(survived)
    
    return training_data, labelled_data

In [58]:
training_data, labelled_data = build_raw_training_dataset()

def train_micrograd():
    multi_layer_perceptron = MLP(5, [16, 16, 1])
    iterations = 100
    learning_rate = -0.0005
    for i in range(0, iterations):
        loss = Value(0.0)

        for j in range(0, 200):
            ypred = multi_layer_perceptron(training_data[j])
            yactual = labelled_data[j]

            loss += (ypred - yactual)**2

            # print(f'{training_data[j]}')
            # print(f'Prediction: {ypred}')
            # print(f'Actual data: {labelled_data[j]}')
            # print(f'Loss: {loss}')
            # print('===================')

        # backward pass
        print(f'{i} => Loss {loss.data / 200.0}')
        print('===================')

        # zero grad
        for p in multi_layer_perceptron.parameters():
            p.grad = 0.0
        
        loss.backward()

        # update weights
        for p in multi_layer_perceptron.parameters():
            p.data += learning_rate * p.grad  

train_micrograd()


0 => Loss 1.8270741051227162
1 => Loss 1.6273164620415321
2 => Loss 1.2075610671109651
3 => Loss 1.1504938139340128
4 => Loss 1.1129476003767713
5 => Loss 1.0870240303273055
6 => Loss 1.066393169956237
7 => Loss 1.0493535894500174
8 => Loss 1.0347999738703288
9 => Loss 1.0230125105295997
10 => Loss 1.0147280934534773
11 => Loss 1.0140425841299645
12 => Loss 1.0288872890053833
13 => Loss 1.082372634497812
14 => Loss 1.1485951077755336
15 => Loss 1.21599900416705
16 => Loss 1.125440729563256
17 => Loss 1.1688840480048754
18 => Loss 1.110713866752725
19 => Loss 1.137313213616565
20 => Loss 1.093804877160707
21 => Loss 1.1122183991835994
22 => Loss 1.0790171649141094
23 => Loss 1.0931531692982082
24 => Loss 1.0663867675129066
25 => Loss 1.0781217204218705
26 => Loss 1.0555741451578202
27 => Loss 1.0658059018065533
28 => Loss 1.0461636294039471
29 => Loss 1.0553567402356823
30 => Loss 1.0378244745267275
31 => Loss 1.0462413895209037
32 => Loss 1.0303199819157272
33 => Loss 1.038119803370182